# Part I — Foundations  
## 2. Python for Django (Practical) — The Django-Developer Toolkit

This chapter is not “learn all Python.” It’s the *exact Python you’ll constantly use*
when building Django apps: environments, imports, modules, data handling, functions,
classes, exceptions, logging, files, and environment variables (settings/secrets).

If you master this chapter, you’ll write Django code faster, debug confidently, and
avoid the most common beginner pain points (import errors, broken environments,
mystery settings bugs, messy data transformations).

---

## 2.0 Learning Outcomes (What you must be able to do)

By the end, you should be able to:

1. Create and manage a clean Python environment for a Django project.
2. Explain the difference between:
   - a module vs a package
   - an import path vs a filesystem path
   - global vs local imports (and why Django sometimes “can’t find” your code)
3. Use Python data structures to represent request data, DB-like records, and
   template contexts.
4. Write functions that accept `*args` / `**kwargs`, and understand why Django uses
   them so often.
5. Understand classes well enough to read and extend Django’s:
   - class-based views
   - forms
   - middleware
6. Handle exceptions correctly, including “fail fast” vs “recover gracefully.”
7. Use logging properly (instead of `print`) in app code.
8. Read/write files safely using `pathlib` and context managers.
9. Use environment variables for configuration and secrets (industry standard).

---

## 2.1 Python Environments (venv) and Dependency Management

### 2.1.1 Why environments exist (the real reason)
If you install packages globally, you quickly get:
- version conflicts between projects
- “works on my machine” issues
- difficulty reproducing setups for teammates/CI/production

A **virtual environment** isolates dependencies per project, like:

```text
Project A uses Django 5.x + requests 2.x
Project B uses Django 4.x + requests 3.x
They don’t break each other because they use separate envs.
```

### 2.1.2 The minimum industry-standard setup
A common baseline (simple and widely used):

- Create a virtual environment in `.venv/`
- Activate it
- Install dependencies with `pip`
- Track dependencies in `requirements.txt` (and optionally `requirements-dev.txt`)

#### Create and activate a venv

```bash
python -m venv .venv
```

Activate:

- macOS/Linux:

```bash
source .venv/bin/activate
```

- Windows PowerShell:

```bash
.\.venv\Scripts\Activate.ps1
```

Verify you’re in the venv:

```bash
python -c "import sys; print(sys.executable)"
```

You should see a path containing `.venv`.

### 2.1.3 Installing packages with pip
Inside the activated venv:

```bash
python -m pip install --upgrade pip
python -m pip install django
```

Why use `python -m pip` instead of `pip`?
- It ensures the `pip` you’re calling belongs to the current `python` interpreter,
  reducing “installed to wrong Python” problems.

### 2.1.4 Capturing dependencies (`requirements.txt`)
To record installed packages:

```bash
python -m pip freeze > requirements.txt
```

Later, to recreate:

```bash
python -m pip install -r requirements.txt
```

#### What `requirements.txt` actually does
It pins specific versions (or ranges, depending on how you generate it), making your
project reproducible.

Example:

```text
Django==5.0.6
asgiref==3.8.1
sqlparse==0.5.1
```

### 2.1.5 “Pinned vs flexible” versions (important concept)
- **Pinned** (`Django==5.0.6`): reproducible, stable builds; safer for teams/CI.
- **Flexible** (`Django>=5.0,<6.0`): allows upgrades; can introduce changes
  unexpectedly if not locked elsewhere.

Industry practice:
- Applications often pin exact versions for production reproducibility.
- Libraries may use version ranges.

You’ll later learn about lockfiles and dependency update workflows; for now, aim for:
- reproducible dev setup
- explicit dependencies

---

## 2.2 Python Packages, Modules, and Imports (Why Django Import Errors Happen)

Django projects are Python projects. Most “Django can’t find X” errors are actually
Python import/package issues.

### 2.2.1 Module vs package (precise definitions)
- **Module**: a single `.py` file you can import.
  - Example: `utils.py` → `import utils`
- **Package**: a directory of modules (usually with `__init__.py`).
  - Example: `myapp/` containing `__init__.py`, `views.py`, `models.py`

Example structure:

```text
project_root/
  myapp/
    __init__.py
    views.py
    utils.py
```

Then:

```python
from myapp import utils
from myapp.utils import some_function
```

### 2.2.2 How Python decides what you can import (sys.path)
Python searches for imports in locations listed in `sys.path`. Typical entries:
- the current working directory (or project root when running Django)
- installed packages in the virtual environment
- standard library

You can inspect:

```python
import sys
print(sys.path)
```

This is why:
- running scripts from random directories can break imports
- Django’s `manage.py` helps by running with the project context

### 2.2.3 Absolute imports vs relative imports
**Absolute import** (most common in Django apps):

```python
from myapp.services.billing import charge_customer
```

**Relative import** (can be useful inside a package):

```python
from .billing import charge_customer
from ..utils import money_to_cents
```

Industry practice in Django apps:
- prefer absolute imports for clarity (especially across apps)
- use relative imports inside tightly-coupled subpackages if you want

### 2.2.4 Common import mistake: naming collisions
If you name your file `django.py`, you can break imports:

```text
django.py  (your file)
```

Then `import django` imports your file, not the real Django package.

Common “don’t name your file this” list:
- `django.py`
- `requests.py`
- `json.py`
- `logging.py`
- `email.py`

These shadow standard/third-party modules.

---

## 2.3 Data Structures You’ll Use Constantly in Django

Django code is mostly:
- mapping keys to values (dict-like)
- iterating lists of objects
- representing structured data in nested shapes

### 2.3.1 Lists (ordered collections)
Use lists for:
- ordered results
- sequences of items
- query results after evaluation

Example:

```python
articles = ["Intro", "Models", "Templates"]
articles.append("Forms")
```

In Django, you might build a template context:

```python
context = {
    "featured_articles": ["Intro", "Models", "Templates"],
}
```

### 2.3.2 Dicts (key/value mappings)
Use dicts for:
- structured records
- configurations
- template contexts
- JSON responses

Example record:

```python
user = {"id": 9, "username": "alice", "is_staff": False}
```

Access:

```python
username = user["username"]
```

Safer access with `.get` (avoids KeyError):

```python
timezone = user.get("timezone", "UTC")
```

In Django, query params often behave dict-like:

```text
/articles/?page=2&sort=price
```

Django later exposes them as a mapping (a `QueryDict`), but the idea is the same:
keys and values.

### 2.3.3 Tuples (fixed-size records, immutable)
Use tuples for:
- coordinates, pairs
- returning multiple values
- constants

Example:

```python
MIN_MAX = (1, 100)
```

### 2.3.4 Sets (unique items)
Use sets for:
- uniqueness checks
- membership tests

Example:

```python
allowed_roles = {"admin", "editor", "viewer"}

if role in allowed_roles:
    ...
```

### 2.3.5 Comprehensions (industry standard for clean transformations)

#### List comprehension
Transform a list:

```python
names = [" Alice ", "Bob", "  Cara"]
clean = [n.strip() for n in names]
```

Filter + transform:

```python
numbers = [1, 2, 3, 4, 5, 6]
evens_squared = [n * n for n in numbers if n % 2 == 0]
```

#### Dict comprehension
Build dicts cleanly:

```python
pairs = [("a", 1), ("b", 2)]
d = {k: v for k, v in pairs}
```

#### Why this matters for Django
You will constantly:
- reshape data for templates
- reshape data for APIs
- build lookup maps (id → object)

Example (common optimization pattern):

```python
users = [
    {"id": 1, "name": "A"},
    {"id": 2, "name": "B"},
]
users_by_id = {u["id"]: u for u in users}
```

Now lookup is fast:

```python
users_by_id[2]  # {"id": 2, "name": "B"}
```

---

## 2.4 Functions: Arguments, `*args`, `**kwargs`, and Why Django Uses Them

### 2.4.1 Functions are “callable units of behavior”
Example:

```python
def greet(name):
    return f"Hello, {name}"
```

### 2.4.2 Default parameters (common in configuration)
```python
def format_price(amount, currency="USD"):
    return f"{currency} {amount:.2f}"
```

### 2.4.3 `*args` and `**kwargs` (critical for Django)
- `*args` collects extra positional arguments into a tuple.
- `**kwargs` collects extra keyword arguments into a dict.

Example:

```python
def demo(*args, **kwargs):
    return {"args": args, "kwargs": kwargs}


result = demo(1, 2, x=10, y=20)
# args -> (1, 2)
# kwargs -> {"x": 10, "y": 20}
```

#### Why Django uses `**kwargs` in routing
When Django matches a URL like:

```text
/articles/42/
```

it may pass captured path variables as keyword arguments:

```python
def article_detail(request, article_id):
    ...
```

Conceptually:

```python
article_detail(request, article_id=42)
```

So your view signature must accept those names.

### 2.4.4 Keyword-only arguments (useful for clarity)
```python
def create_user(username, *, is_staff=False):
    ...
```

Now `is_staff` must be passed by name:

```python
create_user("alice", is_staff=True)
```

This prevents mistakes in function calls—very useful in large codebases.

### 2.4.5 Pure functions vs side effects (quality concept)
- Pure function: output depends only on input; no external changes.
- Side effects: writes files, sends email, hits DB.

Why it matters in Django:
- Pure functions are easy to test.
- Side-effect functions should be separated/controlled (service layer, tasks, etc.).

Example split:

```python
def calculate_total(items):
    return sum(item["price"] for item in items)


def save_invoice_to_db(invoice):
    # side effect
    ...
```

---

## 2.5 Modules and “Running Code Correctly” (`if __name__ == "__main__"`)

### 2.5.1 Why this pattern exists
Python files can be:
- imported (as modules), or
- executed (as scripts)

Example:

```python
def main():
    print("Hello")


if __name__ == "__main__":
    main()
```

When imported, `main()` won’t run automatically. This prevents surprising side
effects.

This matters in Django because:
- Django imports your modules on startup
- you do not want “random code” executing at import time

### 2.5.2 “Import-time side effects” are a common bug
Bad pattern:

```python
# bad_module.py
print("Running at import time!")  # runs whenever imported
```

If Django imports it, your print runs every startup (and can break things if it does
real work like DB calls).

Better pattern:
- define functions/classes
- run logic only in explicit entry points

---

## 2.6 Classes (Enough to Read and Extend Django)

Django uses classes heavily (especially CBVs, forms, middleware).

### 2.6.1 What a class is (practically)
A class bundles:
- data (attributes)
- behavior (methods)

Example:

```python
class Counter:
    def __init__(self):
        self.value = 0

    def inc(self):
        self.value += 1
        return self.value
```

Usage:

```python
c = Counter()
c.inc()  # 1
c.inc()  # 2
```

### 2.6.2 Inheritance (common in Django)
Django provides base classes you extend.

Example idea:

```python
class Animal:
    def speak(self):
        return "..."


class Dog(Animal):
    def speak(self):
        return "woof"
```

Why you care:
- Django class-based views work like this: you override methods/attributes to change
  behavior.

### 2.6.3 Composition over inheritance (often better)
Instead of subclassing everything, you can inject helpers:

```python
class EmailSender:
    def send(self, to, subject, body):
        ...


class SignupService:
    def __init__(self, email_sender):
        self.email_sender = email_sender

    def signup(self, user_email):
        self.email_sender.send(user_email, "Welcome", "Hi!")
```

This is easier to test and often cleaner in large projects.

---

## 2.7 Dataclasses (Clean “Data Containers” for Django-adjacent Code)

Django models are not dataclasses, but dataclasses are useful for:
- service layer inputs/outputs
- structured config objects
- internal DTOs (data transfer objects)

Example:

```python
from dataclasses import dataclass


@dataclass(frozen=True)
class Money:
    amount_cents: int
    currency: str = "USD"

    def format(self) -> str:
        return f"{self.currency} {self.amount_cents / 100:.2f}"
```

Usage:

```python
price = Money(amount_cents=1299)
price.format()  # "USD 12.99"
```

Why `frozen=True`?
- makes it immutable (safer in complex logic)
- prevents accidental changes

---

## 2.8 Typing (Type Hints) for Django Developers (Practical, Not Academic)

Type hints:
- improve editor autocomplete
- catch mistakes earlier (mypy/pyright)
- document intent

### 2.8.1 Basic type hints you’ll actually use

```python
def add(a: int, b: int) -> int:
    return a + b
```

Lists and dicts:

```python
from typing import Any


def summarize(items: list[dict[str, Any]]) -> int:
    return len(items)
```

Optional values:

```python
from typing import Optional


def find_user(username: str) -> Optional[dict[str, str]]:
    if username == "alice":
        return {"username": "alice"}
    return None
```

Why Optional matters:
- forces you to handle `None` cases explicitly

### 2.8.2 Keep typing lightweight early
Industry standard in Django teams varies. Many teams:
- use type hints in service layer and utilities first
- gradually type more critical code
- avoid fighting the framework in early stages

You can still get huge benefits without going “full typing everywhere.”

---

## 2.9 Exceptions: How to Fail Correctly and Debug Faster

### 2.9.1 The difference between “error” and “exception”
In Python, “exceptions” are how errors are represented and propagated.

If an exception is not caught, it stops execution (and in Django, returns 500 in prod,
or a debug page in dev).

### 2.9.2 Catch exceptions only when you can do something meaningful
Bad (swallows real errors):

```python
try:
    do_something()
except Exception:
    pass
```

This hides bugs and makes systems unreliable.

Better (catch specific exceptions and handle them):

```python
try:
    value = int(user_input)
except ValueError:
    value = 0
```

### 2.9.3 Raise your own exceptions to enforce invariants
If something should never happen, say so:

```python
def get_discount(percent: int) -> float:
    if not (0 <= percent <= 100):
        raise ValueError("percent must be between 0 and 100")
    return percent / 100
```

This is how you keep logic correct early and detect issues fast.

### 2.9.4 Common Django style you’ll see later
Django uses exceptions for:
- object not found
- validation errors
- permission errors

You’ll learn their names later, but the pattern is the same:
- raise meaningful exceptions
- handle them at the right layer (view/API boundary)

---

## 2.10 Logging (Use This Instead of `print`)

### 2.10.1 Why `print` is not good enough
- can’t control levels (info/warn/error)
- not structured
- not routed to correct destinations in production
- hard to filter and search

### 2.10.2 Basic logging setup (conceptual)
In application code, you typically do:

```python
import logging

logger = logging.getLogger(__name__)


def process_order(order_id: int) -> None:
    logger.info("Processing order %s", order_id)
```

Important detail:
- use `%s` formatting style; don’t build strings yourself:
  - logging can skip formatting when the level is disabled
  - avoids extra overhead

Bad:

```python
logger.info(f"Processing order {order_id}")
```

Better:

```python
logger.info("Processing order %s", order_id)
```

### 2.10.3 Log levels
- DEBUG: detailed internal info
- INFO: normal events
- WARNING: something unexpected but system continues
- ERROR: operation failed
- CRITICAL: system-wide failure

In Django projects:
- dev often uses DEBUG level
- production often uses INFO/WARNING/ERROR

You’ll configure full Django logging later; for now, understand correct usage in code.

---

## 2.11 File I/O and Paths (Use `pathlib`, Avoid Common Mistakes)

### 2.11.1 Why `pathlib` is preferred
- cross-platform paths (Windows vs Linux)
- cleaner code
- safer composition

Example:

```python
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent
data_path = BASE_DIR / "data" / "users.json"
```

### 2.11.2 Read a text file safely
```python
from pathlib import Path

path = Path("notes.txt")

content = path.read_text(encoding="utf-8")
print(content)
```

### 2.11.3 Write a file safely
```python
from pathlib import Path

path = Path("output.txt")
path.write_text("Hello\n", encoding="utf-8")
```

### 2.11.4 The context manager pattern (what it guarantees)
Equivalent explicit form:

```python
with open("output.txt", "w", encoding="utf-8") as f:
    f.write("Hello\n")
```

Why `with` matters:
- guarantees file is closed even if an exception occurs

### 2.11.5 A security note (important mindset)
Never trust file paths from users blindly.
Later when handling uploads:
- validate filenames
- store safely
- avoid path traversal vulnerabilities (`../../etc/passwd` patterns)

You’ll revisit this in Django file uploads and security chapters.

---

## 2.12 Environment Variables (Configuration the Industry Way)

### 2.12.1 Why environment variables exist
Hardcoding secrets in code is a serious vulnerability.

Bad:

```python
SECRET_KEY = "my-secret-key"
DATABASE_PASSWORD = "password123"
```

Problems:
- leaks in git history
- shared with team unintentionally
- hard to have different configs for dev/staging/prod

Industry standard (12-factor idea):
- config in environment variables
- code is the same across environments

### 2.12.2 Reading env vars in Python
```python
import os

debug = os.environ.get("DEBUG", "false").lower() == "true"
```

If you need a required variable:

```python
import os

secret_key = os.environ["SECRET_KEY"]  # KeyError if missing
```

Why this is good:
- fails fast if misconfigured
- prevents running insecurely by accident

### 2.12.3 `.env` files (dev convenience) — conceptually
A `.env` file is a local file containing:

```text
DEBUG=true
SECRET_KEY=dev-only-secret
```

You generally:
- use it in development
- do not commit real secrets to git
- use real env vars in production

We’ll implement robust settings management later; for now, understand the goal:
- separate configuration from code

---

# 2.13 Hands-On Labs (Do These; They Prevent Months of Confusion)

## Lab 1 — Create a clean Python workspace the “Django way”
Goal: prove you can create a reproducible environment.

1) Create a folder:

```bash
mkdir django-workbook
cd django-workbook
```

2) Create and activate a venv:

```bash
python -m venv .venv
source .venv/bin/activate
```

3) Upgrade pip and install Django:

```bash
python -m pip install --upgrade pip
python -m pip install django
```

4) Save dependencies:

```bash
python -m pip freeze > requirements.txt
```

5) Verify:

```bash
python -c "import django; print(django.get_version())"
```

What this teaches:
- your project is isolated
- dependencies are recorded
- you can reproduce the environment later

Checkpoint question:
- If a teammate clones your repo, what one command installs your dependencies?

Expected answer:
- `python -m pip install -r requirements.txt`

---

## Lab 2 — Build a tiny “config loader” like you’ll use in Django settings
Goal: become comfortable with env vars and defaults.

Create `config_demo.py`:

```python
import os


def get_bool(name: str, default: bool = False) -> bool:
    raw = os.environ.get(name)
    if raw is None:
        return default

    value = raw.strip().lower()
    if value in {"1", "true", "yes", "y", "on"}:
        return True
    if value in {"0", "false", "no", "n", "off"}:
        return False

    raise ValueError(f"Invalid boolean for {name}: {raw!r}")


def main() -> None:
    debug = get_bool("DEBUG", default=False)
    print(f"DEBUG = {debug}")


if __name__ == "__main__":
    main()
```

Run without env var:

```bash
python config_demo.py
```

Now set env var:

```bash
DEBUG=true python config_demo.py
```

Try invalid:

```bash
DEBUG=maybe python config_demo.py
```

What this teaches:
- how real apps avoid silent misconfiguration
- how to validate env vars rather than guessing

---

## Lab 3 — Modules, packages, and imports (fix the “it can’t import my code” problem)
Goal: understand imports the same way Django does.

Create structure:

```text
python_import_lab/
  app/
    __init__.py
    text_tools.py
  run.py
```

Create `app/text_tools.py`:

```python
def slugify(text: str) -> str:
    text = text.strip().lower()
    return "-".join(text.split())
```

Create `run.py`:

```python
from app.text_tools import slugify


def main() -> None:
    print(slugify("  Hello Django World  "))


if __name__ == "__main__":
    main()
```

Run:

```bash
python run.py
```

Now break it intentionally:
- rename `app/` to `app2/` but keep import unchanged
- observe the import error
- fix the import path

What this teaches:
- imports are about package/module names
- Django apps are packages; naming matters

---

## Lab 4 — Logging + exception handling (a realistic utility you’ll reuse)
Goal: log useful information and handle invalid inputs cleanly.

Create `process_users.py`:

```python
import json
import logging
from dataclasses import dataclass
from pathlib import Path

logger = logging.getLogger(__name__)


@dataclass(frozen=True)
class User:
    id: int
    username: str
    is_staff: bool = False


def parse_user(row: dict) -> User:
    try:
        return User(
            id=int(row["id"]),
            username=str(row["username"]).strip(),
            is_staff=bool(row.get("is_staff", False)),
        )
    except KeyError as e:
        raise ValueError(f"Missing required field: {e.args[0]}") from e
    except (TypeError, ValueError) as e:
        raise ValueError(f"Invalid user row: {row!r}") from e


def main() -> None:
    logging.basicConfig(level=logging.INFO)

    input_path = Path("users.json")
    if not input_path.exists():
        logger.error("Input file does not exist: %s", input_path)
        raise SystemExit(1)

    raw = json.loads(input_path.read_text(encoding="utf-8"))
    if not isinstance(raw, list):
        logger.error("Expected a JSON list, got: %s", type(raw).__name__)
        raise SystemExit(1)

    users: list[User] = []
    for row in raw:
        try:
            users.append(parse_user(row))
        except ValueError as e:
            logger.warning("Skipping bad row: %s", e)

    logger.info("Parsed %s users", len(users))

    output_path = Path("clean_users.json")
    output_payload = [
        {"id": u.id, "username": u.username, "is_staff": u.is_staff} for u in users
    ]
    output_path.write_text(
        json.dumps(output_payload, indent=2),
        encoding="utf-8",
    )
    logger.info("Wrote cleaned users to %s", output_path)


if __name__ == "__main__":
    main()
```

Create `users.json`:

```json
[
  { "id": "1", "username": " alice ", "is_staff": true },
  { "id": 2, "username": "bob" },
  { "username": "missing_id" },
  "not a dict"
]
```

Run:

```bash
python process_users.py
```

What to observe:
- It logs INFO/WARNING/ERROR messages with context
- It skips invalid rows instead of crashing immediately
- It still fails fast when critical expectations are violated (file missing, wrong
  JSON top-level type)

Why this is industry-relevant:
- Django apps constantly parse/validate external inputs (requests, webhooks, CSVs)
- good logging + controlled exceptions make production support possible

---

# 2.14 Exercises (Write answers + code)

1) Explain the difference between:
- a package and a module
- `Content-Type` and `Accept` (from previous chapter)
- a pinned vs flexible dependency

2) Write a function:

```python
def clamp(n: int, min_value: int, max_value: int) -> int:
    ...
```

Rules:
- if `n` is below min, return min
- if above max, return max
- otherwise return `n`

3) Write a function that converts query-like inputs:

Input:

```python
params = {"page": "2", "page_size": "50"}
```

Output:

```python
{"page": 2, "page_size": 50}
```

Requirements:
- missing keys use defaults (page=1, page_size=20)
- invalid values raise ValueError with a helpful message
- page_size must be capped at 100

4) Create a small package `tools/` with a module `money.py` and import it from a
script without modifying `sys.path`.

---

# 2.15 Checklist (You should be able to do all of these)

- [ ] Create `.venv`, activate it, install packages, freeze requirements.
- [ ] Explain how imports work and fix a module naming collision.
- [ ] Use dict/list comprehensions to transform data.
- [ ] Write functions using `**kwargs` and explain why frameworks use it.
- [ ] Understand class basics + inheritance enough to read Django CBVs.
- [ ] Catch specific exceptions and raise meaningful errors.
- [ ] Use `logging.getLogger(__name__)` and log levels appropriately.
- [ ] Use `pathlib.Path` for file paths.
- [ ] Read configuration from environment variables safely.

---

## 2.16 Chapter Summary (What to retain permanently)
- Environments make projects reproducible; most setup bugs are env bugs.
- Imports are a naming system, not “file loading”; avoid collisions and import-time
  side effects.
- Data structures + comprehensions are your daily tools for shaping data.
- `*args`/`**kwargs` are everywhere because frameworks pass flexible parameters.
- Logging and exceptions are how production systems stay debuggable.
- Env vars are how real apps manage settings and secrets.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='1. web_dev_essentials.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='3. developer_tooling_and_workflow.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
